<a href="https://colab.research.google.com/github/elifsare/Titanic-Survival-Data/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.engine import input_layer

In [ ]:
def preprocess(data):
  data.Cabin.fillna('0', inplace=True)
  data.loc[data.Cabin.str[0] == 'A', 'Cabin'] = 0
  data.loc[data.Cabin.str[0] == 'B', 'Cabin'] = 1
  data.loc[data.Cabin.str[0] == 'C', 'Cabin'] = 2
  data.loc[data.Cabin.str[0] == 'D', 'Cabin'] = 3
  data.loc[data.Cabin.str[0] == 'E', 'Cabin'] = 4
  data.loc[data.Cabin.str[0] == 'F', 'Cabin'] = 5
  data.loc[data.Cabin.str[0] == 'G', 'Cabin'] = 6
  data.loc[data.Cabin.str[0] == 'T', 'Cabin'] = 7
    
  # Cinsiyeti tam sayıya çevirelim
  data['Sex'].replace('female', 0, inplace=True)
  data['Sex'].replace('male', 1, inplace=True)
    
  # Gemiye biniş limanlarını tam sayıya çevirelim
  data['Embarked'].replace('S', 0, inplace=True)
  data['Embarked'].replace('C', 1, inplace=True)
  data['Embarked'].replace('Q', 2, inplace=True)
    
  # Olmayan (NA) yaş değerlerini medyan ile dolduralım
  data['Age'].fillna(data['Age'].median(), inplace=True)
  data['Fare'].fillna(data['Fare'].median(), inplace=True)
  data['Embarked'].fillna(data['Embarked'].median(), inplace=True)
    
  # İstersek olmayan (NA) değerleri tamamen silebiliriz
  # data.dropna(subset=['Fare', 'Embarked'], inplace=True, how='any')
  return data



In [ ]:
# data içindeki gupların başlıklarıni değiştirme
def group_titles(data):
    data['Names'] = data['Name'].map(lambda x: len(re.split(' ', x)))
    data['Title'] = data['Name'].map(lambda x: re.search(', (.+?) ', x).group(1))
    data['Title'].replace('Master.', 0, inplace=True)
    data['Title'].replace('Mr.', 1, inplace=True)
    data['Title'].replace(['Ms.','Mlle.', 'Miss.'], 2, inplace=True)
    data['Title'].replace(['Mme.', 'Mrs.'], 3, inplace=True)
    data['Title'].replace(['Dona.', 'Lady.', 'the Countess.', 'Capt.', 'Col.', 'Don.', 'Dr.', 'Major.', 'Rev.', 'Sir.', 'Jonkheer.', 'the'], 4, inplace=True)

In [ ]:
def data_subset(data):
    features = ['Pclass', 'SibSp', 'Parch', 'Sex', 'Names', 'Title', 'Age', 'Cabin'] #, 'Fare', 'Embarked']
    lengh_features = len(features)
    subset = data[features]#.fillna(0)
    return subset, lengh_features


In [ ]:
# modelin içinde neler olacak, tasarım
def create_model(train_set_size, input_lenght, num_epochs, batch_size):
  # Define Sequential model with 3 layers
  model = Sequential()
  # Also, let's build a Sequential model incrementally via the add() method
  model.Add(Dense(7, input_dim = input_lenght, activation='softplus')) # dense ----> katman
  model.Add(Dense(3, activation='softplus'))
  model.Add(Dense(1, activation='softplus'))

  lr = .001 # Learning_rate, Defaults to 0.001.
  adam0 = Adam(lr = lr)

  # loss='binary_crossentropy'
  # Entropy is a measure of the uncertainty associated with a given distribution q(y).
  # Binary crossentropy is a loss function that is used in binary classification tasks. These are tasks that answer a question with only two choices. 
  #Several independent such questions can be answered at the same time, as in multi-label classification.
  # Configures the model for training --> model.compile
  model.compile(loss='binary_crossentropy', optimizer = adam0, metrics = ['accuracy'])
  filepath = 'weights.best.hdf5' # öğrenilen ağırlığın kaydı için dosya
  checkpoint = ModelCheckpoint(filepath, monitor = 'acc', verbose = 1, save_best_only = True, mode = 'max') # checkpoint yardımıyla best result'ları belirler
  callbacks_list = [checkpoint]
  
  history_model = model.fit(X_train[:train_set_size], Y_train[:train_set_size], callbacks=callbacks_list, epochs=num_epochs, batch_size=batch_size, verbose=0) #40, 32
  return model, history_model

In [ ]:
def plot(history):
  loss_history = history.history['loss']
  acc_history = history.history['acc']
  epochs = [(i + 1) for i in range(num_epochs)]

  ax = plt.subplot(211)
  ax.plot(epochs, loss_history, color='red')
  ax.set_xlabel('Epochs')
  ax.set_ylabel('Error Rate\n')
  ax.set_title('Error Rate per Epoch\n')

  ax2 = plt.subplot(212)
  ax2.plot(epochs, acc_history, color='blue')
  ax2.set_xlabel('Epochs')
  ax2.set_ylabel('Accuracy\n')
  ax2.set_title('Accuracy per Epoch\n')
  
  plt.subplots_adjust(hspace=0.8)
  plt.savefig('Accuracy_loss.png')
  plt.close()
                                   